# LDA 모델

### 1. 데이터 불러오기

In [22]:
import pandas as pd

# 시 데이터 불러오기
poem = pd.read_csv("../data/nlp_poem.csv", index_col=False)
poem.head()

,author,title,content
0,이순,가을에는 이런 사람이 그립다,수평선이나 지평선 보다<br>더 깊고 아득한 눈빛으로<br>세상의 그리움 다 깨워서...
1,문정희,가을 노트,<br><br><br>그대 떠나간 후<br>나의 가을은<br>조금만 건드려도<br>...
2,이순,가을비 오는 날은,<br>자정이 되어 간신히 그치는 비<br>간신히 버리는 그리움<br>그동안 너무 ...
3,이성선,가을 편지,<br>잎이 떨어지고 있습니다<br>원고지처럼 하늘이 한 칸씩<br>비어 가고 있습...
4,박제영,가을에는,가을에는 잠시 여행을 떠날 일이다<br>그리 수선스러운 준비는 하지 말고<br>그리...


## 2. 데이터 전처리

In [23]:
# 불용어
stopwords = [
    '하다', '이다', '아니다', '않다', '되다', '있다', '없다', '싶다', '알다', '보다', '마다', '같다', '지다', '가다',
    '나', '너', '그대', '당신', '우리', '이', '그', '저', '이런', '저런', '그런', '무엇', '것', '내', 
    '하나', '모든', '때문', '다음', '위해', '사람', '줄', '못', '그리다', '하소', '가장', '많다', '수', '말', '때',
    '음', '갖다', '이기', '사랑', '해도', '더', '땅', '별', '거'
]

In [24]:
from konlpy.tag import Okt

# 토큰화 함수
def tokenize(text):
    okt = Okt()

    # 줄바꿈(<br>) 제거
    text = text.replace("<br>", " ")

    # 형태소(품사) 분석 - 정규화有, 어간추출有
    okt_pos = okt.pos(text, norm=True, stem=True)

    # 특정 품사(명사, 동사, 형용사)만 사용
    lst_word = [word for word, pos in okt_pos if pos in ["Noun"]]

    # 불용어 제거
    lst_word = [word for word in lst_word if word not in stopwords]

    return lst_word

In [36]:
# 시 내용에 토큰화 적용
content = poem["content"]
content_tokens = content.apply(tokenize)
content_tokens.head()

0    [수평선, 지평선, 눈빛, 세상, 그리움, 다른, 유심, 귀, 마음, 날, 폭죽, ...
1    [후, 가을, 조금, 우수수, 몸, 다한, 다한, 노래, 씨앗, 가슴, 지고, 머,...
2    [자정, 비, 그리움, 그동안, 세월, 지나, 짐, 것임, 마침내, 가을비, 날, ...
3    [잎, 원고지, 하늘, 칸, 비어, 빈, 곳, 맑은, 영혼, 잉크, 물, 편지, 함...
4    [가을, 잠시, 여행, 일이, 준비, 데, 가을, 스스로, 푸른, 하늘, 가을, 그...
Name: content, dtype: object

In [37]:
from gensim import corpora

dictionary = corpora.Dictionary(content_tokens)
corpus = [dictionary.doc2bow(text) for text in content_tokens]

In [38]:
import gensim

NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(
    corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15
)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.017*"눈" + 0.013*"속" + 0.012*"날" + 0.011*"손"')
(1, '0.023*"댁" + 0.017*"쪽" + 0.015*"아무" + 0.014*"겨울"')
(2, '0.011*"삶" + 0.010*"낙" + 0.008*"시간" + 0.007*"이제"')
(3, '0.021*"소리" + 0.020*"생각" + 0.016*"얼굴" + 0.013*"바람"')
(4, '0.012*"입술" + 0.011*"속" + 0.010*"심장" + 0.010*"곳"')
(5, '0.028*"길" + 0.023*"시간" + 0.023*"바람" + 0.017*"글자"')
(6, '0.028*"사과" + 0.018*"혼자" + 0.015*"위" + 0.014*"동쪽"')
(7, '0.013*"곳" + 0.012*"렌트" + 0.010*"소리" + 0.008*"계단"')
(8, '0.022*"새" + 0.020*"고양이" + 0.019*"날" + 0.014*"서로"')
(9, '0.016*"속" + 0.016*"거" + 0.015*"알" + 0.012*"길"')


In [39]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

C:\Programming\Miniconda\miniconda3\envs\nlp3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
